In [1]:
from datasets import load_dataset
import json

# Define the datasets to load
datasets_to_load = [
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "spanish",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "french",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "german",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "portuguese",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "italian",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "arabic",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
    {
        "name": "cardiffnlp/tweet_sentiment_multilingual",
        "config_name": "hindi",
        "split": "test",
        "text_field": "text",
        "label_field": "label",
    },
]

In [8]:
# # Function to generate the batch translation request body
# def generate_batch_request(text, index, language):
#     return {
#         "custom_id": f"task-{language}-{index}",
#         "method": "POST",
#         "url": "/v1/chat/completions",
#         "body": {
#             "model": "gpt-4o-mini",
#             "messages": [
#                 {
#                     "role": "system",
#                     "content": "You are a helpful assistant that translates text to English.",
#                 },
#                 {
#                     "role": "user",
#                     "content": f"Please translate the following to English and only the translation: {text}",
#                 },
#             ],
#             "temperature": 0.1,
#         },
#     }


# # Load the datasets and generate batch requests
# for dataset_info in datasets_to_load:
#     dataset = load_dataset(
#         dataset_info["name"],
#         name=dataset_info["config_name"],
#         split=dataset_info["split"],
#     )
#     filename = f"../translation_jobs/batch_cardiff_requests_{dataset_info['config_name']}.jsonl"
#     with open(filename, "w") as f:
#         for i, example in enumerate(dataset):
#             text = example[dataset_info["text_field"]]
#             language = dataset_info["config_name"]
#             request = generate_batch_request(text, i, language)
#             f.write(json.dumps(request) + "\n")

Submit these to openai batch api here - https://platform.openai.com/batches 

Then save the responses in the same the responses with the same name but swapping requests for responses 

e.g. as src/translation_jobs/batch_cardiff_requests_arabic.jsonl --> src/translation_jobs/batch_cardiff_responses_arabic.jsonl


In [10]:
from datasets import load_dataset, DatasetDict
import json


def load_translations(filename):
    translations = []
    with open(filename, "r") as f:
        for line in f:
            response = json.loads(line)
            translation = response["response"]["body"]["choices"][0]["message"][
                "content"
            ]
            translations.append(translation)
    return translations


hf_org = "AIM-Harvard"
all_splits = {}

for dataset_info in datasets_to_load:
    # Load the full dataset for the given config
    full_dataset = load_dataset(
        dataset_info["name"],
        name=dataset_info["config_name"],
    )

    # Select only the test split
    test_dataset = full_dataset["test"]

    # Load translations from your JSONL file
    translation_filename = f"../translation_jobs/batch_cardiff_responses_{dataset_info['config_name']}.jsonl"
    translations = load_translations(translation_filename)

    # Add the translations as a new column
    test_dataset = test_dataset.add_column("translation", translations)

    # Store this test set under a key named by the language/config
    all_splits[dataset_info["config_name"]] = test_dataset

# Create a DatasetDict from all the test splits
dataset_dict = DatasetDict(all_splits)

# Generate a clean name for pushing to the Hub
cleaned_name = datasets_to_load[0]["name"].replace("/", "_")
translated_name = f"{cleaned_name}_translated"

# Push once, containing all languages (one split per language)
dataset_dict.push_to_hub(f"{hf_org}/{translated_name}", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/AIM-Harvard/cardiffnlp_tweet_sentiment_multilingual_translated/commit/f2ad118ce34b4c21d815646554ba70c09f7753d5', commit_message='Upload dataset', commit_description='', oid='f2ad118ce34b4c21d815646554ba70c09f7753d5', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
hf_org = "AIM-Harvard"
df = load_dataset(
    f"{hf_org}/cardiffnlp_tweet_sentiment_multilingual_translated", split="arabic"
)
df[0]

{'text': 'نوال الزغبي (الشاب خالد ليس عالمي) هههههههه أتفرجي على ها الفيديو يا مبتدئة http vía @user',
 'label': 0,
 'translation': 'Nawal El Zoghbi (Cheb Khaled is not my world) hahahaha watch this video, you beginner http via @user'}